<a href="https://colab.research.google.com/github/wjdgoruds2/AI-_assignment/blob/master/Decision%20tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from sklearn import datasets
import numpy as np

def test_split(index,value,dataset):
  left,right=list(),list()
  for row in dataset:
    if row[index]<value:
      left.append(row)
    else:
      right.append(row)
  return left,right

def gini_index(groups,classes):
  n_instances=float(sum([len(group)for group in groups]))
  gini=0.0
  for group in groups:
    size=float(len(group))
    if size==0:
      continue
    score=0.0
    for class_val in classes:
      p=[row[-1]for row in group].count(class_val)/size
      score += p*p
    gini += (1.0-score)*(size/n_instances)
  return gini

def get_split(dataset):
  class_values=list(set(row[-1] for row in dataset))
  b_index,b_value,b_score,b_groups=999,999,999,None
  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups=test_split(index,row[index],dataset)
      gini=gini_index(groups,class_values)
      if gini<b_score:
        b_index,b_value,b_score,b_groups=index,row[index],gini,groups
  return {'index': b_index,'value':b_value,'groups':b_groups}

def split(node,max_depth,min_size,depth):
  left,right=node['groups']
  del(node['groups'])
  if not left or not right:
    node['left']=node['right']=to_terminal(left+right)
    return
  if depth>=max_depth:
    node['left'],node['right']=to_terminal(left),to_terminal(right)
    return
  if len(left) <=min_size:
    node['left']=to_terminal(left)
  else:
    node['left']=get_split(left)
    split(node['left'],max_depth,min_size,depth+1)
  if len(right)<=min_size:
    node['right']=to_terminal(right)
  else:
    node['right']=get_split(right)
    split(node['right'],max_depth,min_size,depth+1)

def to_terminal(group):
  outcomes=[row[-1]for row in group]
  return max(set(outcomes),key=outcomes.count)

def bulid_tree(train,max_depth,min_size):
  root=get_split(train)
  split(root,max_depth,min_size,1)
  return root

def print_tree(node,depth=0):
  if isinstance(node,dict):
    print('%s[X%d < %.3f]'%((depth*'',(node['index']+1),node['value'])))
    print_tree(node['left'],depth+1)
    print_tree(node['right'],depth+1)
  else:
    print('%s[%s]'%((depth*'',node)))

iris=datasets.load_iris()
dataset=np.c_[iris.data,iris.target]

tree=bulid_tree(dataset,3,1)
print_tree(tree)


[X3 < 3.000]
[X1 < 5.100]
[X1 < 4.900]
[0.0]
[0.0]
[X1 < 5.100]
[0.0]
[0.0]
[X4 < 1.800]
[X3 < 5.000]
[1.0]
[2.0]
[X3 < 4.900]
[2.0]
[2.0]
